## Importing necessary libraries and the dataset

In [ ]:
# General stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mapping stuff
!pip install chart_studio  &> /dev/null
import chart_studio.plotly as py
import chart_studio.tools as tl
import plotly.graph_objs as go

#Speed stuff
from numba import jit

In [ ]:
# Loading the Dataset
!gdown --id 1YhGx9t9NccJVmx2gPkCOnb8vHkAsYH82  &> /dev/null
!mkdir dataset  > /dev/null
!unzip grid_data.zip -d dataset  > /dev/null
data_path = '/content/dataset/'

In [ ]:
# Reading the useful csv files
ev_history = pd.read_csv(data_path +'ev_history.csv')
ev_home_locations = pd.read_csv(data_path +'ev_home_locations.csv')
ev_models = pd.read_csv(data_path +'ev_models.csv')
grid_locations = pd.read_csv(data_path +'grid_locations.csv')
gridload = pd.read_csv(data_path +'gridload.csv')
public_chargers_locations = pd.read_csv(data_path +'public_chargers_locations.csv')

In [ ]:
# Calling this cell to drop extra index column 
ev_history.drop(['Unnamed: 0'],axis=1,inplace=True)
ev_home_locations.drop(['Unnamed: 0'],axis=1,inplace=True)
ev_models.drop(['Unnamed: 0'],axis=1,inplace=True)
grid_locations.drop(['Unnamed: 0'],axis=1,inplace=True)
gridload.drop(['Unnamed: 0'],axis=1,inplace=True)
public_chargers_locations.drop(['Unnamed: 0'],axis=1,inplace=True)

## Visualizing the grid and public charger locations

In [ ]:
# Function is used when you have several points, and you want to style them differently for some reason.
# it can later be added to data array of plot_points_to_map() function.
def style_point(size, color, lat_,lon_):
  return go.Scattermapbox(
          lat=lat_,
          lon=lon_,
          mode='markers',
          marker=dict(
              size=size,
              color=color,
              opacity=1
          ),
          hoverinfo='text'
      )

In [ ]:
# A simplified version of code below
def plot_points(data_,mean_lat,mean_lon,loc_label="Tartu",zoom_factor=10):

  # Setting credentials stuff
  tl.set_credentials_file(username='mlguystartu', api_key='27Rx41md4ADPEqWxwgUT')
  mapbox_access_token = "pk.eyJ1IjoibWxndXlzdGFydHUiLCJhIjoiY2tnemthNXRiMTRrdzJ6cjFmc2Jtc3RhNCJ9.xFd2wN_jhsTMxSxO3WAnPw"

  data=data_

  layout = go.Layout(
      title=loc_label,
      autosize=True,
      hovermode='closest',
      showlegend=False,
      mapbox=dict(
          accesstoken=mapbox_access_token,
          bearing=0,
          center=dict(
              lat=mean_lat,
              lon=mean_lon
          ),
          pitch=0,
          zoom=zoom_factor,
          style='light'
      ),
  )
    
  # Generate the figure using the iplot function 
    
  fig = dict(data=data, layout=layout)
  return fig

In [ ]:
###
# Tutorial from https://colab.research.google.com/github/la-counts/data-adventures/blob/master/Instructable_6_How_to_Map_Geographic_Data_Using_Coordinates.ipynb#scrollTo=sxb7cAsfVNuY
#
# Params:
#   site_lon - column for longitude
#   site_lat - column for latitude
#   locations_name - column for locations (one per row)
# zoom_factor(optional) - factor by which we want to zoom the map
###

def plot_points_to_map(site_lon, site_lat, locations_name, loc_label, zoom_factor=10): 
 
  # Setting credentials stuff
  tl.set_credentials_file(username='mlguystartu', api_key='27Rx41md4ADPEqWxwgUT')
  mapbox_access_token = "pk.eyJ1IjoibWxndXlzdGFydHUiLCJhIjoiY2tnemthNXRiMTRrdzJ6cjFmc2Jtc3RhNCJ9.xFd2wN_jhsTMxSxO3WAnPw"

  # Generate the data for the map 
  data = [
      go.Scattermapbox(
          lat=site_lat,
          lon=site_lon,
          mode='markers',
          marker=dict(
              size=3,
              color='rgb(255, 0, 0)',
              opacity=1
          ),
          text=locations_name,
          hoverinfo='text'
      ),
      go.Scattermapbox(
          lat=site_lat,
          lon=site_lon,
          mode='markers',
          marker=dict(
              size=8,
              color='rgb(242, 177, 172)',
              opacity=0.8
          ),
          hoverinfo='none'
      )]


  # Generate a layout around Los Angeles (well, no, it's Tartu), zoomed in so we can see the data points 

  layout = go.Layout(
      title=loc_label,
      autosize=True,
      hovermode='closest',
      showlegend=False,
      mapbox=dict(
          accesstoken=mapbox_access_token,
          bearing=0,
          center=dict(
              lat=np.mean(site_lat),
              lon=np.mean(site_lon)
          ),
          pitch=0,
          zoom=zoom_factor,
          style='light'
      ),
  )
    
  # Generate the figure using the iplot function 
    
  fig = dict(data=data, layout=layout)
  return fig
  

In [ ]:
fig = plot_points_to_map(grid_locations.longitude,grid_locations.latitude \
                         ,grid_locations.address,"Grid locations",zoom_factor=10.5)
py.iplot(fig, filename='Grid locations')

In [ ]:
fig = plot_points_to_map(public_chargers_locations.longitude \
                         ,public_chargers_locations.latitude,public_chargers_locations.address \
                         ,"Public chargers locations",zoom_factor=10.5)
py.iplot(fig, filename='Public chargers locations')

## Analyzing the Dataset

This section is dedicated to analyzing the data obtained and modifying datasets to see the bigger picture and try to mannipulate the data.

### Counting hours spent at a certain location

In [ ]:
ev_home_locations.head()

,address,cadaster,latitude,longitude,x,y
0,"1, Kvissentali põik, Ülejõe, Tartu linn, Tartu...",79514:037:0033,58.406210,26.706300,6477173.28,658139.66
1,"28b/1, Orava, Raadi-Kruusamäe, Tartu linn, Tar...",79512:036:0020,58.389311,26.737114,6475347.92,660042.39
2,"16, Alevi, Karlova, Tartu linn, Tartu, Tartu l...",79508:016:0001,58.362547,26.726161,6472365.82,659521.12
3,"37, Voolu, Variku, Tartu linn, Tartu, Tartu li...",79509:019:0003,58.346618,26.702650,6470513.21,658227.79
4,"12, Ilmatsalu, Veeriku, Tartu linn, Tartu, Tar...",79502:006:0095,58.375785,26.690146,6473716.85,657353.49


In [ ]:
public_chargers_locations.head()

,address,cadaster,latitude,longitude,x,y
0,"78, Tiigi, Kesklinn, Tartu linn, Tartu, Tartu ...",79506:006:0001,58.373603,26.710675,6473552.48,658596.39
1,"30, Veski, Kesklinn, Tartu linn, Tartu, Tartu ...",79507:021:0003,58.378190,26.711309,6474073.17,658582.03
2,"Heino Elleri nimeline Tartu Muusikakool, 15, L...",79507:019:0002,58.380050,26.717865,6474277.29,658997.44
3,"20A, Sepa, Ropka tööstusrajoon, Tartu linn, Ta...",79511:002:0027,58.350656,26.729897,6471029.11,659793.82
4,"7, Tamme põik, Tammelinn, Tartu linn, Tartu, T...",79504:053:0019,58.357376,26.695822,6471706.87,657778.20


In [ ]:
ev_history.head()

,time,model,driving,connected,soc,charge_need,cadaster
0,1,Volkswagen ID.3,False,True,36.250000,7.200000,79514:037:0033
1,2,Volkswagen ID.3,True,False,43.450000,0.000000,79514:037:0033
2,3,Volkswagen ID.3,False,True,40.799133,4.200867,79514:037:0033
3,4,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033
4,5,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033


In [ ]:
gridload.head()

,cadaster,time,baseload
0,79511:003:0022,1,98.997612
1,79511:003:0022,2,104.894207
2,79511:003:0022,3,85.218661
3,79511:003:0022,4,98.941638
4,79511:003:0022,5,79.889267


In [ ]:
grid_locations.head()

,address,cadaster,latitude,longitude,x,y,max_current
0,"2, Aardla, Ropka, Tartu linn, Tartu, Tartu lin...",79511:003:0022,58.355604,26.730697,6471601.92,659795.21,200
1,"66, Peetri, Raadi-Kruusamäe, Tartu linn, Tartu...",79512:038:0009,58.390402,26.730959,6475470.49,659684.48,160
2,"64, Metshaldja, Ihaste, Tartu linn, Tartu, Tar...",79517:014:0024,58.352182,26.777857,6471306.96,662602.18,160
3,"11, Risti, Raadi-Kruusamäe, Tartu linn, Tartu,...",79512:029:0011,58.393000,26.724314,6475718.16,659262.11,60
4,"8, Kristalli, Ränilinn, Tartu linn, Tartu, Tar...",79505:005:0079,58.349762,26.683184,6470844.17,657076.56,160


In [ ]:
ev_models.head()

,models,battery_size,charge_power,efficiency
0,Tesla Model S,72.5,16.5,0.175
1,Nissan Leaf e+,56.0,6.6,0.172
2,Renault Zoe,41.0,22.0,0.164
3,Volkswagen ID.3,45.0,7.2,0.161


In [ ]:
public_chargers_locations_copy = public_chargers_locations.copy()
ev_home_locations_copy = ev_home_locations.copy()

business_chargers = []
for i in public_chargers_locations_copy['cadaster']:
  business_chargers.append(ev_history[ev_history['cadaster'] == i].shape[0])

business_homes = []
for i in ev_home_locations_copy['cadaster']:
  business_homes.append(ev_history[ev_history['cadaster'] == i].shape[0])

In [ ]:
public_chargers_locations_copy['business'] = business_chargers
ev_home_locations_copy['business'] = business_homes

The idea is to find out relations between units around the city and number of hours they are used. For example, we have a cadaster number 79506:006:0001. We know that for public locations, during the day there were two Renault Zoe, 7-18 each. So we can denote that this charger was used for 24 hours in total per day (2 cars, 12 hours each). This way, we may first see how "busy" are locations, and second learn how to match them by cadaster number. 

Here is how we could visualize this information:

In [ ]:
points = []
# Let us find style points for public_chargers and home_locations
for _,i in public_chargers_locations_copy.iterrows():
  points.append(style_point(i['business']/3+1, 'rgb(200,20,20)',[i['latitude']],[i['longitude']]))

for _,i in ev_home_locations_copy.iterrows():
  points.append(style_point(i['business']/3+1, 'rgb(20,20,200)',[i['latitude']],[i['longitude']]))

In [ ]:
fig = plot_points(points,
            np.mean(public_chargers_locations_copy['latitude']),
            np.mean(public_chargers_locations_copy['longitude']))
py.iplot(fig, filename='Chargers')

We could observe that the personal homes are more or less equally have autos, and that is indeed logical, since households have one car each. But for public chargers we can see a trend, where most cars prefer North-West of Tartu. Please note, the information presented only takes into account one day. It describes total number of hours used.

To analyze the model further, we should incorporate the amount of electricity consumed on each point into the calculation of weight (size of a point) and see if the trend changes.

### Adding power consumed to the equation

Perhaps, the number of hours which was spent on a certain location wouldn't be as good as the power spent on specific location. Let us analyze these points with respect to total power. Here we will assume that all the charge that is needed is received from the charger in one hour (which is unrealistic, we don't know how much time exactly did the car spend charging, so we need to estimate this somehow further with respect to ability of car to intake charge):

In [ ]:
charges_per_loc_public = []
for cadaster in  public_chargers_locations_copy['cadaster']:
  charge_per_loc = ev_history[
                              (ev_history['cadaster'] == cadaster)
                               & (ev_history['connected'] == True)
                             ]['charge_need']
  charges_per_loc_public.append(sum(charge_per_loc))

public_chargers_locations_copy['charge_consumed'] = charges_per_loc_public

charges_per_loc_home = []
for cadaster in  ev_home_locations_copy['cadaster']:
  charge_per_loc = ev_history[
                              (ev_history['cadaster'] == cadaster)
                               & (ev_history['connected'] == True)
                             ]['charge_need']
  charges_per_loc_home.append(sum(charge_per_loc))

ev_home_locations_copy['charge_consumed'] = charges_per_loc_home

In [ ]:
points = []

for _,i in public_chargers_locations_copy.iterrows():
  points.append(style_point(i['charge_consumed']/6+1, 'rgb(200,20,20)',[i['latitude']],[i['longitude']]))

for _,i in ev_home_locations_copy.iterrows():
  points.append(style_point(i['charge_consumed']/6+1, 'rgb(20,20,200)',[i['latitude']],[i['longitude']]))

In [ ]:
fig = plot_points(points,
            np.mean(public_chargers_locations_copy['latitude']),
            np.mean(public_chargers_locations_copy['longitude']))
py.iplot(fig, filename='Chargers')

Still, North-West of Tartu shows the same trend to be overloaded all the time. 

## Using Machine learning for prediction

In [ ]:
# Distance function 
@jit
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

### Preparing the dataset for task 1

In [ ]:
all_charger_locations = pd.concat([ev_home_locations,public_chargers_locations],axis = 0).drop(["x","y"],axis=1)

In [ ]:
# Calculating the distance between the charger and the closest grid

grid_cadasters = []
for _,charger in all_charger_locations.iterrows():
  all_distances = [haversine_np(charger['longitude'],charger['latitude'], grid['longitude'], grid['latitude']) for _,grid in grid_locations.iterrows()]
  grid_idx = np.argmin(all_distances)
  grid_cadasters.append(grid_locations.iloc[grid_idx][['cadaster','longitude','latitude']])

all_charger_locations[['closest_grid','grid_longitude','grid_latitude']] = grid_cadasters

In [ ]:
all_charger_locations.head()

,address,cadaster,latitude,longitude,closest_grid,grid_longitude,grid_latitude
0,"1, Kvissentali põik, Ülejõe, Tartu linn, Tartu...",79514:037:0033,58.406210,26.706300,79514:037:0108,26.697076,58.403447
1,"28b/1, Orava, Raadi-Kruusamäe, Tartu linn, Tar...",79512:036:0020,58.389311,26.737114,79512:038:0010,26.731004,58.390288
2,"16, Alevi, Karlova, Tartu linn, Tartu, Tartu l...",79508:016:0001,58.362547,26.726161,79508:020:0022,26.726307,58.360031
3,"37, Voolu, Variku, Tartu linn, Tartu, Tartu li...",79509:019:0003,58.346618,26.702650,79509:019:0001,26.702650,58.346618
4,"12, Ilmatsalu, Veeriku, Tartu linn, Tartu, Tar...",79502:006:0095,58.375785,26.690146,79502:020:0001,26.687757,58.373693


## Visualizing the closest grids to the charger locations on the map

In [ ]:
# 5 colors
red = 'rgb(255,0,0)'
green = 'rgb(0,255,0)'
blue = 'rgb(0,0,255)'
orange = 'rgb(255,105,0)'
magenta = 'rgb(238,31,239)'
pink = 'rgb(150,10,50)'

colors = [red,green,blue,orange,magenta]
points = []


for i,charger in grid_locations.iterrows():
    points.append(style_point(12,pink,[grid_locations.iloc[i]['latitude']],[grid_locations.iloc[i]['longitude']]))


for i,color in enumerate(colors):
  points.append(style_point(8,color,[all_charger_locations.iloc[i]['latitude']],[all_charger_locations.iloc[i]['longitude']]))
  points.append(style_point(12,color,[all_charger_locations.iloc[i]['grid_latitude']],[all_charger_locations.iloc[i]['grid_longitude']]))


fig = plot_points(points,np.mean(all_charger_locations['latitude']),np.mean(all_charger_locations['longitude']),zoom_factor=11)
py.iplot(fig, filename='Chargers')

# Calculate loads per hour for each grid to create training dataset

In [ ]:
gridload.head()

,cadaster,time,baseload
0,79511:003:0022,1,98.997612
1,79511:003:0022,2,104.894207
2,79511:003:0022,3,85.218661
3,79511:003:0022,4,98.941638
4,79511:003:0022,5,79.889267


In [ ]:
ev_history.head()

,time,model,driving,connected,soc,charge_need,cadaster
0,1,Volkswagen ID.3,False,True,36.250000,7.200000,79514:037:0033
1,2,Volkswagen ID.3,True,False,43.450000,0.000000,79514:037:0033
2,3,Volkswagen ID.3,False,True,40.799133,4.200867,79514:037:0033
3,4,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033
4,5,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033


In [ ]:
ev_history_filtered = ev_history.copy()

connected_col = ev_history_filtered['connected']
disconnected_idxs = connected_col.index[connected_col == False].tolist()
ev_history_filtered.drop(disconnected_idxs, inplace=True)

In [ ]:
grid_cadasters = []

for i, entry in ev_history_filtered.iterrows():
    grid_cadaster = all_charger_locations[all_charger_locations['cadaster'] == entry['cadaster']]['closest_grid'].iloc[0]
    grid_cadasters.append(grid_cadaster)

ev_history_filtered['grid_cadaster'] = grid_cadasters

In [ ]:
grid_tmp = []
grid_cadaster_tmp = []
time_tmp = []
totalc_tmp = []
# 0 for lat, 1 for lon
coordinates = [[],[]]
for j, gc in grid_locations.iterrows():
  for i in range(1, 25):
    grid_tmp.append(j)
    grid_cadaster_tmp.append(gc['cadaster'])
    time_tmp.append(i)
    coordinates[0].append(gc['latitude'])
    coordinates[1].append(gc['longitude'])
    totalc_tmp.append(np.sum(ev_history_filtered[(ev_history_filtered['time'] == i) & (ev_history_filtered['grid_cadaster'] == gc['cadaster'])]['charge_need']))

In [ ]:
dataset = pd.DataFrame()
dataset['time'] = time_tmp
dataset['grid'] = grid_tmp
dataset['latitude'] = coordinates[0]
dataset['longitude'] = coordinates[1]
dataset['total_charge'] = totalc_tmp
dataset['grid_cadaster'] = grid_cadaster_tmp

dataset.head()

,time,grid,latitude,longitude,total_charge,grid_cadaster
0,1,0,58.355604,26.730697,0.0,79511:003:0022
1,2,0,58.355604,26.730697,0.0,79511:003:0022
2,3,0,58.355604,26.730697,0.0,79511:003:0022
3,4,0,58.355604,26.730697,0.0,79511:003:0022
4,5,0,58.355604,26.730697,0.0,79511:003:0022


In [ ]:
dataset.describe()

,time,grid,latitude,longitude,total_charge
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,12.500000,24.500000,58.367714,26.726592,4.914399
std,6.925073,14.436886,0.017859,0.029212,9.930169
min,1.000000,0.000000,58.343786,26.681245,0.000000
25%,6.750000,12.000000,58.353122,26.707049,0.000000
50%,12.500000,24.500000,58.359886,26.723493,0.000000
75%,18.250000,37.000000,58.383861,26.732628,6.600000
max,24.000000,49.000000,58.403447,26.789745,87.200000


# TASK 1, to predict load for a grid/hour

In [ ]:
dataset_modified = dataset.copy()
base_loads = []
max_loads = []
total_cons = []

for i, row in dataset_modified.iterrows():
  base = gridload[(gridload['cadaster'] == row['grid_cadaster']) & (gridload['time'] == row['time'])]['baseload'].iloc[0]
  max_ = grid_locations[(grid_locations['cadaster'] == row['grid_cadaster'])]['max_current'].iloc[0]
  base_loads.append(base)
  max_loads.append(max_)
  total_cons.append(base + row['total_charge'])

dataset_modified['base_load'] = base_loads
dataset_modified['max_load'] = max_loads
dataset_modified['actual_load'] = total_cons

dataset_modified.head()

,time,grid,latitude,longitude,total_charge,grid_cadaster,base_load,max_load,actual_load
0,1,0,58.355604,26.730697,0.0,79511:003:0022,98.997612,200,98.997612
1,2,0,58.355604,26.730697,0.0,79511:003:0022,104.894207,200,104.894207
2,3,0,58.355604,26.730697,0.0,79511:003:0022,85.218661,200,85.218661
3,4,0,58.355604,26.730697,0.0,79511:003:0022,98.941638,200,98.941638
4,5,0,58.355604,26.730697,0.0,79511:003:0022,79.889267,200,79.889267


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

X = dataset_modified[['time', 'longitude', 'latitude', 'base_load', 'max_load']]
y = dataset_modified['actual_load']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor()
gbrt = GradientBoostingRegressor()
hgbrt = HistGradientBoostingRegressor()

rf.fit(X_train, y_train)
gbrt.fit(X_train, y_train)
hgbrt.fit(X_train, y_train)

# Attribute score returns the R2 Score
print(f"RandomForestRegressor: {rf.score(X_test, y_test)}\n")
print(f"GradientBoostingRegressor: {gbrt.score(X_test, y_test)}\n")
print(f"HistGradientBoostingRegressor: {hgbrt.score(X_test, y_test)}\n")

RandomForestRegressor: 0.9481725417798074

GradientBoostingRegressor: 0.9621860787368463

HistGradientBoostingRegressor: 0.9648008760734397



## Using Grid Search to tune Histogram-based / Gradient Boosting Regression Tree

In [ ]:
# Tuning Gradient Boosting Regression Tree
from sklearn.model_selection import GridSearchCV

param_grid = {
              "learning_rate": [0.1, 0.01, 0.001],
              "n_estimators": [100, 200, 500],
              "max_depth": [3, 5, 10, 20],
              "random_state": [None, 42],
              "warm_start": [False, True]
}

grbt = GradientBoostingRegressor()

grid_search = GridSearchCV(estimator=gbrt, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2) 

grid_search.fit(X_train, y_train)

best_grid = grid_search.best_estimator_

print("")
print(f"The R2 score for GradientBoostingRegressor is: {best_grid.score(X_test, y_test) * 100} %") 

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.3min



The R2 score for GradientBoostingRegressor is: 96.24063409391387 %


[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.8min finished


In [ ]:
# Histogram Gradient Boosting Tree

param_grid = {
              "l2_regularization": [0.0, 0.1, 0.2, 0.3],
              "learning_rate": [0.1, 0.2, 0.3],
              "max_iter": [100, 200],
              "max_depth": [None, 100],
              "random_state": [None, 42],
              "warm_start": [False, True]
}

hgrbt = HistGradientBoostingRegressor()

grid_search = GridSearchCV(estimator=hgbrt, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2) 

grid_search.fit(X_train, y_train)

grid_search.best_params_

# Testing the Tuned HGBRT

best_grid = grid_search.best_estimator_

print("")
print(f"The R2 score for HistGradientBoostingRegressor is: {best_grid.score(X_test, y_test) * 100} %") 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 192 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:  2.1min finished



The R2 score for HistGradientBoostingRegressor is: 96.44016729774211 %


In [ ]:
# Checking whether we could use this model for backend
# best_grid.predict(np.array(X_train.iloc[0]).reshape(1, -1))

## Save the tuned model using joblib for Backend

In [ ]:
from google.colab import files
import joblib

filename = "grid_load_predictor.sav"
joblib.dump(best_grid, filename)

# Uncomment this stuff to download the model directly
# files.download("/content/grid_load_predictor.sav")

['grid_load_predictor.sav']

In [ ]:
# To load the model 

task1 = joblib.load(filename)
task1.score(X_test, y_test)

0.9644016729774211

# TASK 2, SMART CARS AND CHARGERS 

Task: Develop a model for smart charging – a model that divides the electricity consumption of EVs such that the limits of the grid won’t be exceeded and all of the EV owners receive the similar experience.

So here we could do the following for the overloaded grid:


*   Check by how much is it overloaded, e.g. 10 kW
*   Get all the chargers connected to this grid
*   Predict the charge need on them and car model
*   Optimize using this data




In [ ]:
all_charger_locations.head()

,address,cadaster,latitude,longitude,closest_grid,grid_longitude,grid_latitude
0,"1, Kvissentali põik, Ülejõe, Tartu linn, Tartu...",79514:037:0033,58.406210,26.706300,79514:037:0108,26.697076,58.403447
1,"28b/1, Orava, Raadi-Kruusamäe, Tartu linn, Tar...",79512:036:0020,58.389311,26.737114,79512:038:0010,26.731004,58.390288
2,"16, Alevi, Karlova, Tartu linn, Tartu, Tartu l...",79508:016:0001,58.362547,26.726161,79508:020:0022,26.726307,58.360031
3,"37, Voolu, Variku, Tartu linn, Tartu, Tartu li...",79509:019:0003,58.346618,26.702650,79509:019:0001,26.702650,58.346618
4,"12, Ilmatsalu, Veeriku, Tartu linn, Tartu, Tar...",79502:006:0095,58.375785,26.690146,79502:020:0001,26.687757,58.373693


In [ ]:
ev_history_filtered.head()

,time,model,driving,connected,soc,charge_need,cadaster,grid_cadaster
0,1,Volkswagen ID.3,False,True,36.250000,7.200000,79514:037:0033,79514:037:0108
2,3,Volkswagen ID.3,False,True,40.799133,4.200867,79514:037:0033,79514:037:0108
3,4,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033,79514:037:0108
4,5,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033,79514:037:0108
5,6,Volkswagen ID.3,False,True,45.000000,0.000000,79514:037:0033,79514:037:0108


In [ ]:
chargers_trainset = ev_history_filtered.copy()
chargers_trainset.drop(['driving', 'connected', 'grid_cadaster'], axis=1, inplace=True)
chargers_trainset['is_home_loc'] = chargers_trainset.cadaster.isin(ev_home_locations.cadaster).astype(bool)

# sklearn doesn't support strings, so encode the EV models as so:
car_models = {
'Tesla Model S': 0,
'Nissan Leaf e+' : 1,
'Renault Zoe' : 2,
'Volkswagen ID.3' : 3  
}

temp_lat = []
temp_lon = []
temp_mo_num = []
temp_charge_power = []
temp_battery_size = []

for i, row in chargers_trainset.iterrows():
  car_id = car_models[row['model']]
  used_charger = all_charger_locations[all_charger_locations['cadaster'] == row['cadaster']].iloc[0]
  temp_lat.append(used_charger['latitude'])
  temp_lon.append(used_charger['longitude'])
  temp_mo_num.append(car_id)
  car_data = ev_models.iloc[car_id]
  temp_charge_power.append(car_data['charge_power'])
  temp_battery_size.append(car_data['battery_size'])

chargers_trainset['latitude'] = temp_lat
chargers_trainset['longitude'] = temp_lon
chargers_trainset['model_numeric'] = temp_mo_num
chargers_trainset["is_home_loc"] = chargers_trainset["is_home_loc"] * 1
chargers_trainset['battery_size'] = temp_battery_size
chargers_trainset['charge_power'] = temp_charge_power


chargers_trainset.head()

,time,model,soc,charge_need,cadaster,is_home_loc,latitude,longitude,model_numeric,battery_size,charge_power
0,1,Volkswagen ID.3,36.250000,7.200000,79514:037:0033,1,58.40621,26.7063,3,45.0,7.2
2,3,Volkswagen ID.3,40.799133,4.200867,79514:037:0033,1,58.40621,26.7063,3,45.0,7.2
3,4,Volkswagen ID.3,45.000000,0.000000,79514:037:0033,1,58.40621,26.7063,3,45.0,7.2
4,5,Volkswagen ID.3,45.000000,0.000000,79514:037:0033,1,58.40621,26.7063,3,45.0,7.2
5,6,Volkswagen ID.3,45.000000,0.000000,79514:037:0033,1,58.40621,26.7063,3,45.0,7.2


## Model 0: Predicting the car model connected to the charger at a given time

Classifying a model first helps to improve further predictions.



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import HistGradientBoostingClassifier

X = chargers_trainset[['time', 'longitude', 'latitude', 'is_home_loc']]
y = chargers_trainset['model_numeric']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rfm = RandomForestClassifier()
gbrt = GradientBoostingClassifier()
hgbrt = HistGradientBoostingClassifier()

clf1 = LogisticRegression(solver="newton-cg", random_state=42)
clf2 = GradientBoostingClassifier()
clf3 = SVC(probability=True, gamma="scale", random_state=42)
clf4 = GaussianNB()

voting_soft = VotingClassifier(estimators=[('lr', clf1), ('gbrt', clf2), ('gnb', clf4)], voting='soft')
voting_hard = VotingClassifier(estimators=[('lr', clf1), ('gbrt', clf2), ('gnb', clf4)], voting='hard')

rfm.fit(X_train, y_train)
gbrt.fit(X_train, y_train)
hgbrt.fit(X_train, y_train)
voting_soft.fit(X_train, y_train)
voting_hard.fit(X_train, y_train)


print(f"RandomForestClassifier: {rfm.score(X_test, y_test)}\n")
print(f"GradientBoostingClassifier: {gbrt.score(X_test, y_test)}\n")
print(f"HistGradientBoostingClassifier: {hgbrt.score(X_test, y_test)}\n")
print(f"VotingClassifier with Soft voting: {voting_soft.score(X_test, y_test)}\n")
print(f"VotingClassifier with Hard voting: {voting_hard.score(X_test, y_test)}\n")

RandomForestClassifier: 0.7432432432432432

GradientBoostingClassifier: 0.7871621621621622

HistGradientBoostingClassifier: 0.7533783783783784

VotingClassifier with Soft voting: 0.7871621621621622

VotingClassifier with Hard voting: 0.41216216216216217



In [ ]:
_# Tuning Gradient Boosting Classifier

from sklearn.model_selection import GridSearchCV

param_grid = {
              "learning_rate": [0.1, 0.2, 0.15],

              "max_depth": [3, 5, 7, 10]
}

grbt = GradientBoostingClassifier()

grid_search = GridSearchCV(estimator=gbrt, param_grid=param_grid) 

grid_search.fit(X_train, y_train)

best_grid_car = grid_search.best_estimator_

print("")
print(f"Mean Accuracy GradientBoostingClassifier: {best_grid_car.score(X_test, y_test) * 100} %") 


Mean Accuracy GradientBoostingClassifier: 80.06756756756756 %


In [ ]:
# To Download Model 0: Car model predictor

from google.colab import files
import joblib

filename = "car_model_predictor.sav"
joblib.dump(best_grid_car, filename)

# Uncomment this stuff to download the model directly
# files.download("/content/car_model_predictor.sav")

['car_model_predictor.sav']

# Model 1: Predicting SOC of the car 

We are predicting soc because predicting charge need has accuracy of about 30%.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

X = chargers_trainset[['time', 'longitude', 'latitude', 'is_home_loc', 'battery_size', 'charge_power']]
y = chargers_trainset['soc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rfs = RandomForestRegressor()
gbrt = GradientBoostingRegressor()
hgbrt = HistGradientBoostingRegressor()

rfs.fit(X_train, y_train)
gbrt.fit(X_train, y_train)
hgbrt.fit(X_train, y_train)

print(f"RandomForestRegressor: {rfs.score(X_test, y_test)}\n")
print(f"GradientBoostingRegressor: {gbrt.score(X_test, y_test)}\n")
print(f"HistGradientBoostingRegressor: {hgbrt.score(X_test, y_test)}\n")

RandomForestRegressor: 0.7391373494335747

GradientBoostingRegressor: 0.7142003620805752

HistGradientBoostingRegressor: 0.7322326331790331



In [ ]:
# Histogram Gradient Boosting Tree

param_grid = {
              "l2_regularization": [0.0, 0.1, 0.2, 0.3],
              "learning_rate": [0.1, 0.2, 0.3],
              "max_iter": [100, 200],
              "max_depth": [None, 100],
              "random_state": [None, 42],
              "warm_start": [False, True]
}

hgrbt = HistGradientBoostingRegressor()

grid_search = GridSearchCV(estimator=hgbrt, param_grid=param_grid) 

grid_search.fit(X_train, y_train)

grid_search.best_params_

# Testing the Tuned HGBRT

best_grid = grid_search.best_estimator_

print("")
print(f"The R2 score for HistGradientBoostingRegressor is: {best_grid.score(X_test, y_test) * 100} %") 


The R2 score for HistGradientBoostingRegressor is: 73.37956980546763 %


In [ ]:
# Random Forest Tree

param_grid = {
              "n_estimators": [100, 150, 200],
              "max_depth": [None, 10, 20],
              "warm_start": [False, True]
}

rfs = RandomForestRegressor()

grid_search2 = GridSearchCV(estimator=rf, param_grid=param_grid) 

grid_search2.fit(X_train, y_train)

grid_search2.best_params_

# Testing the Tuned HGBRT

best_grid2 = grid_search2.best_estimator_

print("")
print(f"The R2 score for RandomForestRegressor is: {best_grid2.score(X_test, y_test) * 100} %") 


The R2 score for RandomForestRegressor is: 73.69630939223207 %


In [ ]:
# To download Model 1: SOC predictor

from google.colab import files
import joblib

filename = "soc_predictor.sav"
joblib.dump(hgbrt, filename) # Use hgbrt as it's more consistent (R2 score)
 
# Uncomment this stuff to download the model directly
# files.download("/content/soc_predictor.sav")

## Model 2: Predicting the consumption on a charger at a given time

**Not used because this can actually be calcualted.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

X = chargers_trainset[['time', 'longitude', 'latitude', 'soc']]
y = chargers_trainset['charge_need']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rfc = RandomForestRegressor()
gbrt = GradientBoostingRegressor()
hgbrt = HistGradientBoostingRegressor()

rfc.fit(X_train, y_train)
gbrt.fit(X_train, y_train)
hgbrt.fit(X_train, y_train)

print(f"RandomForestRegressor: {rfc.score(X_test, y_test)}\n")
print(f"GradientBoostingRegressor: {gbrt.score(X_test, y_test)}\n")
print(f"HistGradientBoostingRegressor: {hgbrt.score(X_test, y_test)}\n")

RandomForestRegressor: 0.7866483492408616

GradientBoostingRegressor: 0.7755378885638127

HistGradientBoostingRegressor: 0.7854759642608506



In [ ]:
# To download Model 2: Consumption Predictor

from google.colab import files
import joblib

filename = "consumption_predictor.sav"
joblib.dump(hgbrt, filename)

# Uncomment this stuff to download the model directly
# files.download("/content/consumption_predictor.sav")

['consumption_predictor.sav']

# Whole pipeline testing

In [ ]:
from sklearn.metrics import r2_score, max_error, mean_absolute_error

test_set = chargers_trainset.copy()
test_set['pred_model_num'] = best_grid_car.predict(test_set[['time', 'longitude', 'latitude', 'is_home_loc']])

tmp_bat = []
tmp_cp = []

for i, row in test_set.iterrows():
  predicted_model = row['pred_model_num']
  tmp_bat.append(ev_models.iloc[int(predicted_model)]['battery_size'])
  tmp_cp.append(ev_models.iloc[int(predicted_model)]['charge_power'])


test_set['pred_bat_size'] = tmp_bat
test_set['pred_charge_power'] = tmp_cp
test_set['pred_soc'] = rfs.predict(test_set[['time', 'longitude', 'latitude', 'is_home_loc', 'pred_bat_size', 'pred_charge_power']])

tmp_need = []

for i, row in test_set.iterrows():
  tmp_need.append(min(row['pred_bat_size'] - row['pred_soc'], row['pred_charge_power']))

test_set['pred_charge_need'] = tmp_need
test_set['bad_ml'] = rfc.predict(test_set[['time', 'longitude', 'latitude', 'pred_soc']])

print('\n Calculating charge need using predicted soc\n')
print('R2: ', r2_score(test_set[['charge_need']], test_set[['pred_charge_need']]))
print('Max error: ',max_error(test_set[['charge_need']], test_set[['pred_charge_need']]))
print('Mean abs error: ',mean_absolute_error(test_set[['charge_need']], test_set[['pred_charge_need']]))

print('\n Predicting charge need using ML\n')
print('R2: ',r2_score(test_set[['charge_need']], test_set[['bad_ml']]))
print('Max error: ',max_error(test_set[['charge_need']], test_set[['bad_ml']]))
print('Mean abs error: ',mean_absolute_error(test_set[['charge_need']], test_set[['bad_ml']]))



 Calculating charge need using predicted soc

R2:  0.5974711762524749
Max error:  19.51
Mean abs error:  1.821389829361714

 Predicting charge need using ML

R2:  0.29557103307310706
Max error:  19.51
Mean abs error:  3.0856675645948566


**Our models for the second task haven't actually seen the data where a charger has no car connected which should make the results a bit worse.**

# **Some backend prototyping**

In [ ]:
# find all the chargers connected to a certain grid

gc_dataset = grid_locations.copy()
gc_dataset.drop(['address', 'latitude', 'longitude', 'x', 'y', 'max_current'], axis=1, inplace=True)
chargers = []

for i, row in gc_dataset.iterrows():
  grid_chargers = all_charger_locations[all_charger_locations['closest_grid'] == row['cadaster']]
  # basically all pairs of charger cadaster/charge need for this grid at this time
  out = [charger['cadaster'] for _, charger in grid_chargers.iterrows()]
  chargers.append(out)

gc_dataset['chargers'] = chargers